In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [ ]:
spark = SparkSession.builder\
    .appName('safinmr_project_task2')\
    .config("mapreduce.fileoutputcommitter.algorithm.version","2")\
    .master("local[*]")\
    .getOrCreate()

Сгенерировать DataFrame из трёх колонок (row_id, discipline, season) - олимпийские дисциплины по сезонам.
row_id - число порядкового номера строки;
discipline - наименование олимпиский дисциплины на английском (полностью маленькими буквами);
season - сезон дисциплины (summer / winter);
Укажите не мнее чем по 5 дисциплин для каждого сезона.
Сохраните DataFrame в csv-файл, разделитель колонок табуляция, первая строка должна содержать название колонок.
Данные должны быть сохранены в виде 1 csv-файла а не множества маленьких.


In [ ]:
sports_df = spark.createDataFrame(
    [
        (1, 'Handball', 'summer'),
        (2, 'Football', 'summer'),
        (3, 'Swimming', 'summer'),
        (4, 'Voleyball', 'summer'),
        (5, 'Basketball', 'summer'),
        (6, 'Hockey', 'winter'),
        (7, 'Figure skating', 'winter'),
        (8, 'Skeleton', 'winter'),
        (9, 'Bobsled', 'winter'),
        (10, 'Snowboarding', 'winter'),
    ],
    schema='row_id int, discipline string, season string'
)

In [ ]:
sports_df.show()

In [ ]:
sports_df.rdd.getNumPartitions()

In [ ]:
sports_df = sports_df.coalesce(1)

In [ ]:
sports_df.rdd.getNumPartitions()

In [ ]:
sports_df.write\
    .format("csv")\
    .option("delimiter", "\t")\
    .option("header", "true")\
    .mode("overwrite")\
    .save("file:///media/sf_edu/neoflex/project/files/output/sports.csv")

Прочитайте исходный файл "Athletes.csv".
Посчитайте в разрезе дисциплин сколько всего спортсменов в каждой из дисциплин принимало участие.
Результат сохраните в формате parquet.

In [ ]:
athletes_df = spark.read\
    .format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .option("delimiter", ";")\
    .load("file:///media/sf_edu/neoflex/project/files/Athletes.csv")

In [ ]:
athletes_df.show()

In [ ]:
sportsmen_count_df = athletes_df.groupby(F.col("Discipline")).count()
sportsmen_count_df.show()

In [ ]:
sportsmen_count_df.write\
    .format("parquet")\
    .mode("overwrite")\
    .save("file:///media/sf_edu/neoflex/project/files/output/sportsmen_count.parquet")

Прочитайте исходный файл "Athletes.csv".
Посчитайте в разрезе дисциплин сколько всего спортсменов в каждой из дисциплин принимало участие.
Получившийся результат нужно объединить с сгенерированным вами DataFrame из 1-го задания и в итоге вывести количество участников, только по тем дисциплинам, что есть в вашем сгенерированном DataFrame.
Результат сохраните в формате parquet.


In [ ]:
selected_sportsmen_count_df = sports_df.join(
    sportsmen_count_df,
    "discipline",
    "inner"
).select(
    F.col("discipline"),
    F.col("count")
)

In [ ]:
selected_sportsmen_count_df.show()

In [ ]:
selected_sportsmen_count_df.write\
    .format("parquet")\
    .mode("overwrite")\
    .save("file:///media/sf_edu/neoflex/project/files/output/selected_sportsmen_count.parquet")

In [ ]:
spark.stop()